In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import math
import itertools
from IPython.display import clear_output
import numpy as np

%matplotlib inline 
#plt.set_cmap(cm.get_cmap("Spectral_r"))

Transposing cartesian coordinates to latitude / longitude 

https://math.stackexchange.com/questions/296794/finding-the-transform-matrix-from-4-projected-points-with-javascript/339033#339033


| var | cartesian | GPS |
|-----|-----------|-------------------------|
| x1  | (0,0)     | (40.750885, -73.998169) |
| x2  | (0,28)    | (40.768575, -73.985212) |
| x3  | (10,0)    | (40.743195, -73.979931) |
| x4  | (10,28)   | (40.760918, -73.966995) |


Below are just helpful calculations for getting the final combined matrix that transforms from source to destination

In [ ]:
xC1 = (0,0)
xC2 = (0,28)
xC3 = (10,0)
xC4 = (10,28)

xLL1 = (40.750885, -73.998169)
xLL2 = (40.768575, -73.985212)
xLL3 = (40.743195, -73.979931)
xLL4 = (40.760918, -73.966995)

m_AC = np.mat([[xC1[0], xC2[0], xC3[0]], [xC1[1], xC2[1], xC3[1]], [1, 1, 1]])
m_BC = np.array(list(xC4) + [1]).reshape(-1,1)
lambda_C = np.matmul(np.linalg.inv(m_AC), m_BC)
lambda_C_ext = np.repeat(np.transpose(lambda_C), repeats=3,axis=0)
transform_C =  np.multiply(lambda_C_ext, m_AC)

m_ALL = np.mat([[xLL1[0], xLL2[0], xLL3[0]], [xLL1[1], xLL2[1], xLL3[1]], [1, 1, 1]])
m_BLL = np.array(list(xLL4) + [1]).reshape(-1,1)
lambda_LL = np.matmul(np.linalg.inv(m_ALL), m_BLL)
lambda_LL_ext = np.repeat(np.transpose(lambda_LL), repeats=3,axis=0)
transform_LL =  np.multiply(lambda_LL_ext, m_ALL)

fullmap = transform_LL * np.linalg.inv(transform_C)

def get_gps_coords(x, y):
    m_coords = np.transpose(np.mat([x, y, 1]))
    m_output = np.matmul(fullmap, m_coords)
    lat = np.around(m_output[0,0] / m_output[2,0], decimals=6)
    long = np.around(m_output[1,0] / m_output[2,0], decimals=6)
    return (lat, long)

#TEST
tests = [get_gps_coords(*xC1) == xLL1,
get_gps_coords(*xC2) == xLL2,
get_gps_coords(*xC3) == xLL3,
get_gps_coords(*xC4) == xLL4]

print(all(tests))

In [ ]:
cart_Xs = [0, 2, 4, 6, 7, 8, 9, 10]
name_Xs = dict(zip(cart_Xs, ["9","8","7","6","5","Madison","Park","Lexington","3"]))
cart_Ys = list(range(0,29))
name_Ys = dict(zip(cart_Ys, [str(i+30) for i in cart_Ys]))

In [ ]:
DG = nx.DiGraph()
coords = list(itertools.product(cart_Xs,cart_Ys))
for nodeid, pair in enumerate(coords):
    intersection = name_Xs[pair[0]] + "_" +  name_Ys[pair[1]]
    gps = get_gps_coords(*pair)
    DG.add_node(nodeid, cartesian=pair, name = intersection, latitude = gps[0], longitude = gps[1], nodetype="inner", color="#e1e3f0", volume_rate=dict())

nodes = {v: k for k, v in nx.get_node_attributes(DG, "cartesian").items()}

for pair in nodes:
    
    nextave = cart_Xs.index(pair[0])+1
    if nextave >= len(cart_Xs):
        right = (-1,-1)
    else:
        right = (cart_Xs[nextave] , pair[1])
    above = (pair[0], pair[1] + 1)

    eastboundst = (pair[1] % 2) == 0   
    
    if above in nodes:#  and pair[0] not in {1,6}:
        
        if pair[0] in [2, 6, 8, 10]:
            DG.add_edge(nodes[pair], nodes[above], direction='north', lanes = 4, name = name_Xs[pair[0]] + "_ave")
        elif pair[0] == 9:
            DG.add_edge(nodes[pair], nodes[above], direction='north', lanes = 2, name = name_Xs[pair[0]] + "_ave")
            DG.add_edge(nodes[above], nodes[pair], direction='south', lanes = 2, name = name_Xs[pair[0]] + "_ave")
        else:
            DG.add_edge(nodes[above], nodes[pair], direction='south', lanes = 4, name = name_Xs[pair[0]] + "_ave")
            
    if right in nodes: # and pair[1] not in {1,6}:
        if pair[1] in [4, 12, 27]:
            DG.add_edge(nodes[pair], nodes[right], direction='east', lanes = 2, name = name_Ys[pair[1]] + "_st")
            DG.add_edge(nodes[right], nodes[pair], direction='west', lanes = 2, name = name_Ys[pair[1]] + "_st")
        if eastboundst:
            DG.add_edge(nodes[pair], nodes[right], direction='east', lanes = 2, name = name_Ys[pair[1]] + "_st")
        else:
            DG.add_edge(nodes[right], nodes[pair], direction='west', lanes = 2, name = name_Ys[pair[1]] + "_st")
            

for node in DG.nodes:
    border_west = DG.nodes[node]["cartesian"][0] == 0
    border_east = DG.nodes[node]["cartesian"][0] == 10
    
    border_north = DG.nodes[node]["cartesian"][1] == 0
    border_south = DG.nodes[node]["cartesian"][1] == 28
    
    border = (border_west or border_east or border_north or border_south)
    
    if DG.nodes[node]["cartesian"][0] == 0: # west border
        if DG.nodes[node]["cartesian"][1] in [4, 12, 27]:
            DG.nodes[node]["nodetype"] = "source_sink"
            DG.nodes[node]["color"] = "yellow"
        elif len([i for i in DG.out_edges(node) if DG.edges[i]['direction'] == 'east']) > 0:
            DG.nodes[node]["nodetype"] = "source"
            DG.nodes[node]["color"] = "green"
        else:
            DG.nodes[node]["nodetype"] = "sink"
            DG.nodes[node]["color"] = "red"
    elif DG.nodes[node]["cartesian"][0] == 10: # east border
        if DG.nodes[node]["cartesian"][1] in [4, 12, 27]:
            DG.nodes[node]["nodetype"] = "source_sink"
            DG.nodes[node]["color"] = "yellow"
        elif len([i for i in DG.out_edges(node) if DG.edges[i]['direction'] == 'west']) > 0:
            DG.nodes[node]["nodetype"] = "source"
            DG.nodes[node]["color"] = "green"
        else:
            DG.nodes[node]["nodetype"] = "sink"
            DG.nodes[node]["color"] = "red"
    elif DG.nodes[node]["cartesian"][1] == 28: # north border
        if DG.nodes[node]["cartesian"][0] == 9:
            DG.nodes[node]["nodetype"] = "source_sink"
            DG.nodes[node]["color"] = "yellow"
        elif len([i for i in DG.out_edges(node) if DG.edges[i]['direction'] == 'south']) > 0:
            DG.nodes[node]["nodetype"] = "source"
            DG.nodes[node]["color"] = "green"
        else:
            DG.nodes[node]["nodetype"] = "sink"
            DG.nodes[node]["color"] = "red"                
    elif DG.nodes[node]["cartesian"][1] == 0: # south border
        if DG.nodes[node]["cartesian"][0] == 9:
            DG.nodes[node]["nodetype"] = "source_sink"
            DG.nodes[node]["color"] = "yellow"
        elif len([i for i in DG.out_edges(node) if DG.edges[i]['direction'] == 'north']) > 0:
            DG.nodes[node]["nodetype"] = "source"
            DG.nodes[node]["color"] = "green"
        else:
            DG.nodes[node]["nodetype"] = "sink"
            DG.nodes[node]["color"] = "red"        
    
    
    #if border:
     #   print(DG.nodes[node]["name"])
        #DG.nodes[node]["nodetype"] = "source"
        #DG.nodes[node]["color"] = "green"
#     elif border:
#         DG.nodes[node]["nodetype"] = "sink"
#         DG.nodes[node]["color"] = "red"

In [ ]:
mins = [i*60 for i in list(range(24))]
rate = [int(i) for i in list(np.ones(24)*2)]

defaultrate = dict(zip(mins,rate))

for node in DG.nodes:
    if "source" in DG.nodes[node]["nodetype"]:
        DG.nodes[node]["volume_rate"] = defaultrate

In [ ]:
mins = [i*60 for i in list(range(24))]
rate = [int(i) for i in list(np.ones(24)*2)]

defaultrate = dict(zip(mins,rate))
defaultrate

In [ ]:
pos = nx.get_node_attributes(DG,'cartesian')
nodecol = nx.get_node_attributes(DG, 'color').values()
nodename = nx.get_node_attributes(DG, 'name')

plt.figure(figsize=(10,20)) 
nx.draw(DG, 
        pos, 
        width=2,
        node_color=nodecol,
        font_color='blue',
        node_size=500,
        with_labels=True)
    
stnames = nx.get_edge_attributes(DG,"name")
for key in stnames:
    stnames[key] = stnames[key][:5]
    
#nx.draw_networkx_edge_labels(DG,pos,edge_labels=stnames,font_color='black')

plt.show()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pprint
#from GraphTest import DG
import time

# Dict of all paths from every source to every sink.
# This gives all the routes. It is a dict that maps
# (source, sink): [list of routes in ascending order of size]

#Min path size - we can adjust this
MIN_PATH_SIZE = 5


def filter_edge(edge, node, sink):
    if DG.nodes[sink]['cartesian'][0]-DG.nodes[node]['cartesian'][0] < 0 and DG.edges[edge]['direction'] == 'west':
        return True
    if DG.nodes[sink]['cartesian'][0]-DG.nodes[node]['cartesian'][0] > 0 and DG.edges[edge]['direction'] == 'east':
        return True
    if DG.nodes[sink]['cartesian'][1]-DG.nodes[node]['cartesian'][1] < 0 and DG.edges[edge]['direction'] == 'south':
        return True
    if DG.nodes[sink]['cartesian'][1]-DG.nodes[node]['cartesian'][1] > 0 and DG.edges[edge]['direction'] == 'north':
        return True
    if (DG.nodes[sink]['cartesian'][0]-DG.nodes[node]['cartesian'][0] == 0 and 
        DG.nodes[sink]['cartesian'][1]-DG.nodes[node]['cartesian'][1] < 0 and
        DG.edges[edge]['direction'] == 'south'):
        return True
    if (DG.nodes[sink]['cartesian'][0]-DG.nodes[node]['cartesian'][0] == 0 and 
        DG.nodes[sink]['cartesian'][1]-DG.nodes[node]['cartesian'][1] > 0 and
        DG.edges[edge]['direction'] == 'north'):
        return True
    if (DG.nodes[sink]['cartesian'][1]-DG.nodes[node]['cartesian'][1] == 0 and 
        DG.nodes[sink]['cartesian'][0]-DG.nodes[node]['cartesian'][0] < 0 and
        DG.edges[edge]['direction'] == 'west'):
        return True
    if (DG.nodes[sink]['cartesian'][1]-DG.nodes[node]['cartesian'][1] == 0 and 
        DG.nodes[sink]['cartesian'][0]-DG.nodes[node]['cartesian'][0] > 0 and
        DG.edges[edge]['direction'] == 'east'):
        return True
    return False                                           

def discover_paths(node, sink, pair, visited, cur_path, paths):        
        # Mark the current node as visited and push it to the path
        visited[node] = True
        cur_path.append(node)

        # If we have reached the sink, store the path in paths
        if node == sink:
            new_path = []
            for n in cur_path:
                new_path.append(n)
            if len(new_path) >= MIN_PATH_SIZE:
                paths[pair].append(new_path)
        # Otherwise, continue DFS for unvisited neighbors
        else:
            for edge in DG.out_edges(node):
                nbh = edge[1]
                if filter_edge(edge, node, sink):
                    if visited[nbh] is False:
                        discover_paths(nbh, sink, pair, visited, cur_path, paths)

        # If we have finished exploring the node, set it to unvisited and
        # remove it from cur_path so that we can use it again for future paths
        visited[node] = False
        cur_path.pop()


In [ ]:
paths = dict()
s = 2
t = 207
pair = (s,t)
paths[pair] = []
# List storing the current path from source to sink
cur_path = []
# Visited dict for nodes
visited = dict()
for node in DG.nodes:
    visited[node] = False
# Generate all possible paths
discover_paths(s, t, pair, visited, cur_path, paths)
paths[pair].sort(key=lambda x : len(x))
print(pair)

In [ ]:
import json

with open("test.json", 'w') as outf:
    jsonformat = json.dumps({"%s|%s" % k:v for k, v in paths.items()})
    outf.write(jsonformat)

In [ ]:
with open("test.json", "r") as inf:
    data = json.load(inf)
    pairdict = {tuple([int(i) for i in k.split("|")]): v for k, v in data.items()}
    

In [ ]:
import datetime
datetime.date(2021,5,15) - datetime.date(2021,4,4)

In [ ]:
json.dumps({"%s|%s" % k:v for k, v in paths.items()})

In [ ]:
for i, n in enumerate(DG.edges):
    print(n)

In [ ]:
filt = set(itertools.chain.from_iterable(paths[pair]))

In [ ]:
pos = nx.get_node_attributes(DG.subgraph(list(filt)),'cartesian')
nodecol = nx.get_node_attributes(DG.subgraph(list(filt)), 'color').values()
nodename = nx.get_node_attributes(DG.subgraph(list(filt)), 'name')

plt.figure(figsize=(15,5)) 
nx.draw(DG.subgraph(list(filt)), 
        pos, 
        width=2,
        node_color=nodecol,
        font_color='blue',
        node_size=500,
        with_labels=True)
    
stnames = nx.get_edge_attributes(DG.subgraph(list(filt)),"name")
for key in stnames:
    stnames[key] = stnames[key][:5]
    
nx.draw_networkx_edge_labels(DG.subgraph(list(filt)),pos,edge_labels=stnames,font_color='black')

plt.show()

In [ ]:
with open("microsimulation/sumoDataGeneration/data/pathdict.json", 'r') as inf:
    paths = json.load(inf)
    paths = {tuple([int(i) for i in k.split("|")]): v for k, v in paths.items()}

In [ ]:
[i for i in paths if i['path_id' == 1]]

In [ ]:
import os
import pandas

path = os.path.join(os.getcwd(),"microsimulation","Lambdas","lambdas.csv")
df = pandas.read_csv(path)

In [ ]:
df.interval.unique()

In [ ]:
df.groupby(['hour','minute']).agg({'node_id':"nunique"}).shape

In [ ]:
df = df.drop(['road_name','segment_start','segment_end','direction','interval'],axis=1)

In [ ]:
df.lambda_car = df.lambda_car * 5

In [ ]:
df = df.groupby(['node_id','year','month','day','hour','minute']).mean()['lambda_car'].to_frame().reset_index()

In [ ]:
df.lambda_car = df.lambda_car.fillna(0).astype(int)

In [ ]:
df[df.node_id == 202].sort_values("lambda_car")

In [ ]:
coords = pandas.DataFrame([[i, DG.nodes[i]['cartesian'][0], DG.nodes[i]['cartesian'][1]] for i in DG.nodes if 'source' in DG.nodes[i]['nodetype']], columns=['node_id','x','y'])

In [ ]:
coords

In [ ]:
coords.loc[coords.x == 0,'edge'] = "West"
coords.loc[coords.x == 10,'edge'] = "East"
coords.loc[coords.y == 0,'edge'] = "South"
coords.loc[coords.y == 28,'edge'] = "North"

In [ ]:
coords = coords.set_index('node_id')

In [ ]:
df = df.set_index('node_id')

In [ ]:
#pandas.__version__

In [ ]:
coords.loc[coords.index.difference(df.index)]

In [ ]:
coords = coords.reset_index()

In [ ]:
test = coords.set_index(['x','y'])

In [ ]:
df = coords.join(df, how='left')

In [ ]:
df = df.reset_index()

In [ ]:
df

In [ ]:
df['missing'] = df['year'].isnull().astype(int)

In [ ]:
temp = df[['node_id','x','y','edge','missing']].drop_duplicates()

In [ ]:
temp.groupby('edge').agg({"missing":['sum','count']})

In [ ]:
df.to_csv("visual.csv")

In [ ]:
import pandas

In [ ]:
import os

In [ ]:
file = r"/Users/hsajer3/NYU/bdad/BDAD_Violet_Noise/microsimulation/sumoDataGeneration/data/my_route.rou.xml"

In [ ]:
import xml.etree.ElementTree as ET
tree = ET.parse(file)
root = tree.getroot()

In [ ]:
busses = []
cars = []
for i in root.iter("vehicle"):
    if i.attrib["type"] == "Bus":
        busses.append(i.attrib)
    else:
        cars.append(i.attrib)

In [ ]:
busses = pandas.DataFrame.from_records(busses)

In [ ]:
routes = []
for i in root.iter("route"):
    routes.append(i.attrib)

In [ ]:
routes = pandas.DataFrame.from_records(routes)

In [ ]:
routes = routes.rename(columns={"id":"route"})

In [ ]:
busrte = busses.merge(routes,how='inner')[['route','edges']].drop_duplicates()

In [ ]:
import re
busrte.edges = busrte.edges.apply(lambda s: re.split(" ",s))

In [ ]:
busrte['start'] = busrte.edges.apply(lambda s: s[0].replace("Lexington","Lexingtn").split('to')[0])

In [ ]:
busrte['end'] = busrte.edges.apply(lambda s: s[-1].replace("Lexington","Lexingtn").split('to')[1])

In [ ]:
nodes = pandas.DataFrame.from_records([[i, DG.nodes[i]['name']] for i in DG.nodes],columns=["node_id","node_name"])

In [ ]:
nodes.node_name = nodes.node_name.apply(lambda s: s.replace("Lexington","Lexingtn"))

In [ ]:
busrte = busrte.merge(nodes.rename(columns={"node_name":"start"}),how='left').rename(columns={"node_id":"start_id"})

In [ ]:
busrte = busrte.merge(nodes.rename(columns={"node_name":"end"}),how='left').rename(columns={"node_id":"end_id"})

In [ ]:
#busrte.sort_values(['start_id'])

In [ ]:
import json

In [ ]:
file2 = r"/Users/hsajer3/NYU/bdad/BDAD_Violet_Noise/microsimulation/sumoDataGeneration/data/busschedule.json"

In [ ]:
with open(file2,"r") as ins:
    f = json.loads(ins.read())

In [ ]:
stops = set(pandas.DataFrame.from_records(f)['stops'].sum())

In [ ]:
stopmap = dict()
for i, v in enumerate(stops):
    stopmap[v] = "station%s" % i
    print("<busStop id=\"station%s\" lane=\"%s\"/>" % (i,v))

In [ ]:
buspath = pandas.DataFrame.from_records(f)
buspath.stops = buspath.stops.apply(lambda s: [stopmap[i] for i in s])

In [ ]:
with open(r"/Users/hsajer3/NYU/bdad/BDAD_Violet_Noise/microsimulation/sumoDataGeneration/data/busschedule2.json", "w") as out:
    out.write(buspath.to_json(orient='records'))

In [ ]:
with open(file2,"r") as ins:
    f = json.loads(ins.read())

In [ ]:
#tuple([int(i) for i in k.split("|")]): v for k, v in busroute.items()
 
for record in f:
    key = record['busroute']
    record['busroute'] = tuple([int(i) for i in key.split("|")])

In [ ]:
with open(r"/Users/hsajer3/NYU/bdad/BDAD_Violet_Noise/microsimulation/sumoDataGeneration/data/pathdict.json", 'r') as inf:
    paths = json.load(inf)
    paths = {tuple([int(i) for i in k.split("|")]): v for k, v in paths.items()}

In [ ]:
vehicle_id = 0
vehicle_assigns = []
for hour in range(24):
    for bus in f:
        if hour in bus['active_hrs']:
            options = paths[bus['busroute']]
            path_id = [i["path_id"] for i in options if i["turns"] <=1][0]
            vehicle_id += 1
            minute = bus['start_min'] + hour*60
            vehiclestr = '<vehicle depart="{}" id="veh{}" route="route{}" type="{}">\n'.format(minute, vehicle_id, path_id, "Bus")
            stopstr = "".join(["<stop busStop=\"%s\" duration=\"20\"/>\n" % i for i in bus['stops']])
            vehiclestr = vehiclestr + stopstr + "</vehicle>\n"
            
            vehicle_assigns.append(assignment)

In [ ]:
    <vehicle id="0" type="BUS" depart="0" color="1,1,0">
        <route edges="2/0to2/1 2/1to1/1 1/1to1/2 1/2to0/2 0/2to0/1 0/1to0/0 0/0to1/0 1/0to2/0 2/0to2/1"/>
        <stop busStop="busstop1" duration="20"/>
        <stop busStop="busstop2" duration="20"/>
        <stop busStop="busstop3" duration="20"/>
        <stop busStop="busstop4" duration="20"/>
    </vehicle>

In [ ]:
import pickle
with open("/Users/hsajer3/NYU/bdad/BDAD_Violet_Noise/microsimulation/sumoDataGeneration/data/graph.p", "rb") as f:
    DG = pickle.load(f)

In [ ]:
DG.nodes[0]

In [ ]:
["Car1"]*2

In [ ]:
rates = pandas.read_csv("visual.csv")

In [ ]:
rates.loc[rates['simgrouping'].isnull(),"simgrouping"] = rates.loc[rates['simgrouping'].isnull(),"node_id"].astype(str).apply(lambda s: "sgrp_"+s)

In [ ]:
rates = rates[['node_id','simgrouping','hour','minute','lambda_car']]

In [ ]:
idgrp = rates[['node_id','simgrouping']].drop_duplicates()

In [ ]:
rates = rates.groupby(['node_id','simgrouping','hour','minute'],as_index=False).mean()

In [ ]:
rates.loc[:,['hour','minute']] = rates.loc[:,['hour','minute']].astype(int)

In [ ]:
rates.hour = rates.hour - 1

In [ ]:
rates['totalmin'] = rates.hour * 60 + rates.minute

In [ ]:
smooth = rates.groupby(['simgrouping','totalmin']).agg({"lambda_car":"mean"})

In [ ]:
smooth = smooth.reset_index().sort_values('totalmin')

In [ ]:
smooth['rollinglambda'] = smooth.groupby('simgrouping')['lambda_car'].transform(lambda x: x.rolling(4, 1).mean())

In [ ]:
smooth.simgrouping.unique()

In [ ]:
smooth[smooth.simgrouping == 'sgrp_222'][['totalmin','rollinglambda']].plot.line(x='totalmin',y='rollinglambda')

In [ ]:
smooth

In [ ]:
thing = smooth.drop(['lambda_car'],axis=1).set_index('totalmin').groupby('simgrouping').apply(lambda s: s.to_dict(orient='index'))

In [ ]:
import math
smooth.rollinglambda = smooth.rollinglambda.apply(lambda s: round(s,2))

In [ ]:
lambdamap = dict()

for i in smooth['simgrouping'].unique():
    lambdamap[i] = smooth[smooth.simgrouping == i].set_index('totalmin')['rollinglambda'].to_dict()

In [ ]:
nodelambdas = idgrp.set_index('node_id')['simgrouping'].to_dict()

In [ ]:
for k in nodelambdas:
    nodelambdas[k] = lambdamap[nodelambdas[k]]

In [ ]:
check = set(nodelambdas.keys())

In [ ]:
cmp = set([i for i in DG.nodes if "source" in DG.nodes[i]["nodetype"]])

In [ ]:
#nodelambdas

In [ ]:
outp = r"/Users/hsajer3/NYU/bdad/BDAD_Violet_Noise/microsimulation/sumoDataGeneration/data/nodelambdas.json"

In [ ]:
with open(outp,"w") as outs:
    outs.write(json.dumps(nodelambdas))

In [ ]:
with open(outp, "r") as test:
    f = json.load(test)

In [ ]:
f = {int(k):v for k, v in f.items()}

In [ ]:
with open(file2,"r") as ins:
    f = json.load(ins)

In [ ]:
stopmap

In [ ]:
for rec in f:
    rec['stops'] = [stopmap[i] for i in rec['stops']]

In [ ]:
with open(file2, "w") as outs:
    outs.write(json.dumps(f))